In [1]:
from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
d = d.withColumn('m1',substring(d.mac,1,1)).\
      withColumn('m2',substring(d.mac,2,1)).\
      withColumn('m3',substring(d.mac,3,1)).\
      withColumn('m4',substring(d.mac,4,1)).\
      withColumn('m5',substring(d.mac,5,1)).\
      withColumn('m6',substring(d.mac,6,1)).\
      withColumn('m7',substring(d.mac,7,1)).\
      withColumn('m8',substring(d.mac,8,1)).\
      withColumn('m9',substring(d.mac,9,1)).\
      withColumn('m10',substring(d.mac,10,1)).\
      withColumn('m11',substring(d.mac,11,1)).\
      withColumn('m12',substring(d.mac,12,1))  

In [2]:
d.show(truncate=False)

+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|mac         |id   |m1 |m2 |m3 |m4 |m5 |m6 |m7 |m8 |m9 |m10|m11|m12|
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|B8C74A11FB4D|10533|B  |8  |C  |7  |4  |A  |1  |1  |F  |B  |4  |D  |
|D4503F4A9BEB|40209|D  |4  |5  |0  |3  |F  |4  |A  |9  |B  |E  |B  |
|14C697EA51C7|42209|1  |4  |C  |6  |9  |7  |E  |A  |5  |1  |C  |7  |
|386EA2DEC715|41158|3  |8  |6  |E  |A  |2  |D  |E  |C  |7  |1  |5  |
|40CD7A210159|48883|4  |0  |C  |D  |7  |A  |2  |1  |0  |1  |5  |9  |
|E4B2FBA9777E|39226|E  |4  |B  |2  |F  |B  |A  |9  |7  |7  |7  |E  |
|587A6AFB7819|42210|5  |8  |7  |A  |6  |A  |F  |B  |7  |8  |1  |9  |
|208B37F4F60A|10535|2  |0  |8  |B  |3  |7  |F  |4  |F  |6  |0  |A  |
|B0E5EDCCDF22|40780|B  |0  |E  |5  |E  |D  |C  |C  |D  |F  |2  |2  |
|C0F4E69F8772|10803|C  |0  |F  |4  |E  |6  |9  |F  |8  |7  |7  |2  |
|449EF91BE94D|39367|4  |4  |9  |E  |F  |9  |1  |B  |E  |9  |4  |D  |
|9C2EA1C7D9C1|39781|9  |C  |2  |E 

In [3]:
tmp = d.groupBy('id').count()
tmp = tmp.withColumnRenamed('count','c')

In [4]:
tmp.sort('c',ascending=True).show(truncate=False)

+-----+---+
|id   |c  |
+-----+---+
|12259|1  |
|12311|1  |
|12500|1  |
|39629|1  |
|11624|1  |
|12404|1  |
|10149|1  |
|41436|1  |
|12285|1  |
|10358|1  |
|12326|1  |
|10452|1  |
|12167|1  |
|12354|1  |
|12216|1  |
|12397|1  |
|12408|1  |
|12523|1  |
|12512|1  |
|12409|1  |
+-----+---+
only showing top 20 rows



In [5]:
tmp.select('c').summary().show()

+-------+------------------+
|summary|                 c|
+-------+------------------+
|  count|              4128|
|   mean| 49418.97747093023|
| stddev|228834.72165307103|
|    min|                 1|
|    25%|                91|
|    50%|               552|
|    75%|              4030|
|    max|           4435867|
+-------+------------------+



In [6]:
use = tmp.filter(tmp.c > 40000)

In [7]:
use.count(),use.distinct().count()

(470, 470)

In [8]:
use_df = use.repartition(1)
use_pandas = use_df.toPandas()

In [9]:
use_pandas.head()

,id,c
0,39005,539648
1,39492,769767
2,39512,851884
3,42208,1099617
4,41098,270884


In [10]:
use_id = list(use_pandas.id)

In [11]:
def filt(s):
    for x in use_id:
        if x in s:
            return 1
    else:
            return 0

In [12]:
ft = udf(lambda x:filt(x))

In [13]:
d_tmp = d.withColumn('filt',ft('id'))
df = d_tmp.filter(d_tmp.filt == 1).select('mac','id','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12')

In [14]:
df.show(truncate=False)

+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|mac         |id   |m1 |m2 |m3 |m4 |m5 |m6 |m7 |m8 |m9 |m10|m11|m12|
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|B8C74A11FB4D|10533|B  |8  |C  |7  |4  |A  |1  |1  |F  |B  |4  |D  |
|D4503F4A9BEB|40209|D  |4  |5  |0  |3  |F  |4  |A  |9  |B  |E  |B  |
|14C697EA51C7|42209|1  |4  |C  |6  |9  |7  |E  |A  |5  |1  |C  |7  |
|386EA2DEC715|41158|3  |8  |6  |E  |A  |2  |D  |E  |C  |7  |1  |5  |
|40CD7A210159|48883|4  |0  |C  |D  |7  |A  |2  |1  |0  |1  |5  |9  |
|E4B2FBA9777E|39226|E  |4  |B  |2  |F  |B  |A  |9  |7  |7  |7  |E  |
|587A6AFB7819|42210|5  |8  |7  |A  |6  |A  |F  |B  |7  |8  |1  |9  |
|208B37F4F60A|10535|2  |0  |8  |B  |3  |7  |F  |4  |F  |6  |0  |A  |
|B0E5EDCCDF22|40780|B  |0  |E  |5  |E  |D  |C  |C  |D  |F  |2  |2  |
|C0F4E69F8772|10803|C  |0  |F  |4  |E  |6  |9  |F  |8  |7  |7  |2  |
|449EF91BE94D|39367|4  |4  |9  |E  |F  |9  |1  |B  |E  |9  |4  |D  |
|9C2EA1C7D9C1|39781|9  |C  |2  |E 

In [15]:
df.select('id').distinct().count()

470

In [16]:
df.count(),df.distinct().count()

(194068115, 194068115)

In [17]:
df.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('m1', 'string'),
 ('m2', 'string'),
 ('m3', 'string'),
 ('m4', 'string'),
 ('m5', 'string'),
 ('m6', 'string'),
 ('m7', 'string'),
 ('m8', 'string'),
 ('m9', 'string'),
 ('m10', 'string'),
 ('m11', 'string'),
 ('m12', 'string')]

In [18]:
df.show(5,truncate=False)

+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|mac         |id   |m1 |m2 |m3 |m4 |m5 |m6 |m7 |m8 |m9 |m10|m11|m12|
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
|B8C74A11FB4D|10533|B  |8  |C  |7  |4  |A  |1  |1  |F  |B  |4  |D  |
|D4503F4A9BEB|40209|D  |4  |5  |0  |3  |F  |4  |A  |9  |B  |E  |B  |
|14C697EA51C7|42209|1  |4  |C  |6  |9  |7  |E  |A  |5  |1  |C  |7  |
|386EA2DEC715|41158|3  |8  |6  |E  |A  |2  |D  |E  |C  |7  |1  |5  |
|40CD7A210159|48883|4  |0  |C  |D  |7  |A  |2  |1  |0  |1  |5  |9  |
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [19]:
df = df.withColumn('f1',conv(df.m1, 16, 10)).withColumn('f2',conv(df.m2, 16, 10)).\
        withColumn('f3',conv(df.m3, 16, 10)).withColumn('f4',conv(df.m4, 16, 10)).\
        withColumn('f5',conv(df.m5, 16, 10)).withColumn('f6',conv(df.m6, 16, 10)).\
        withColumn('f7',conv(df.m7, 16, 10)).withColumn('f8',conv(df.m8, 16, 10)).\
        withColumn('f9',conv(df.m9, 16, 10)).withColumn('f10',conv(df.m10, 16, 10)).\
        withColumn('f11',conv(df.m11, 16, 10)).withColumn('f12',conv(df.m12, 16, 10))

In [20]:
df.show()

+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|         mac|   id| m1| m2| m3| m4| m5| m6| m7| m8| m9|m10|m11|m12| f1| f2| f3| f4| f5| f6| f7| f8| f9|f10|f11|f12|
+------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|B8C74A11FB4D|10533|  B|  8|  C|  7|  4|  A|  1|  1|  F|  B|  4|  D| 11|  8| 12|  7|  4| 10|  1|  1| 15| 11|  4| 13|
|D4503F4A9BEB|40209|  D|  4|  5|  0|  3|  F|  4|  A|  9|  B|  E|  B| 13|  4|  5|  0|  3| 15|  4| 10|  9| 11| 14| 11|
|14C697EA51C7|42209|  1|  4|  C|  6|  9|  7|  E|  A|  5|  1|  C|  7|  1|  4| 12|  6|  9|  7| 14| 10|  5|  1| 12|  7|
|386EA2DEC715|41158|  3|  8|  6|  E|  A|  2|  D|  E|  C|  7|  1|  5|  3|  8|  6| 14| 10|  2| 13| 14| 12|  7|  1|  5|
|40CD7A210159|48883|  4|  0|  C|  D|  7|  A|  2|  1|  0|  1|  5|  9|  4|  0| 12| 13|  7| 10|  2|  1|  0|  1|  5|  9|
|E4B2FBA9777E|39226|  E|  4|  B|  2|  F|  B|  A|  9|  7|  7|  7|

In [21]:
df.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('m1', 'string'),
 ('m2', 'string'),
 ('m3', 'string'),
 ('m4', 'string'),
 ('m5', 'string'),
 ('m6', 'string'),
 ('m7', 'string'),
 ('m8', 'string'),
 ('m9', 'string'),
 ('m10', 'string'),
 ('m11', 'string'),
 ('m12', 'string'),
 ('f1', 'string'),
 ('f2', 'string'),
 ('f3', 'string'),
 ('f4', 'string'),
 ('f5', 'string'),
 ('f6', 'string'),
 ('f7', 'string'),
 ('f8', 'string'),
 ('f9', 'string'),
 ('f10', 'string'),
 ('f11', 'string'),
 ('f12', 'string')]

In [22]:
df = df.select('mac','id',col('f1').cast('float'),col('f2').cast('float')\
               ,col('f3').cast('float'),col('f4').cast('float')\
               ,col('f5').cast('float'),col('f6').cast('float')\
               ,col('f7').cast('float'),col('f8').cast('float')\
               ,col('f9').cast('float'),col('f10').cast('float')\
               ,col('f11').cast('float'),col('f12').cast('float'))

In [23]:
from pyspark.ml.feature import VectorAssembler
vec = VectorAssembler(inputCols=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12'],outputCol='features')
new_df = vec.transform(df)

In [24]:
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol='id', outputCol='label')
si_model = si.fit(new_df)
df_final = si_model.transform(new_df)

In [25]:
df_final.show(5,truncate=False)

+------------+-----+----+---+----+----+----+----+----+----+----+----+----+----+-------------------------------------------------------+-----+
|mac         |id   |f1  |f2 |f3  |f4  |f5  |f6  |f7  |f8  |f9  |f10 |f11 |f12 |features                                               |label|
+------------+-----+----+---+----+----+----+----+----+----+----+----+----+----+-------------------------------------------------------+-----+
|B8C74A11FB4D|10533|11.0|8.0|12.0|7.0 |4.0 |10.0|1.0 |1.0 |15.0|11.0|4.0 |13.0|[11.0,8.0,12.0,7.0,4.0,10.0,1.0,1.0,15.0,11.0,4.0,13.0]|2.0  |
|D4503F4A9BEB|40209|13.0|4.0|5.0 |0.0 |3.0 |15.0|4.0 |10.0|9.0 |11.0|14.0|11.0|[13.0,4.0,5.0,0.0,3.0,15.0,4.0,10.0,9.0,11.0,14.0,11.0]|107.0|
|14C697EA51C7|42209|1.0 |4.0|12.0|6.0 |9.0 |7.0 |14.0|10.0|5.0 |1.0 |12.0|7.0 |[1.0,4.0,12.0,6.0,9.0,7.0,14.0,10.0,5.0,1.0,12.0,7.0]  |22.0 |
|386EA2DEC715|41158|3.0 |8.0|6.0 |14.0|10.0|2.0 |13.0|14.0|12.0|7.0 |1.0 |5.0 |[3.0,8.0,6.0,14.0,10.0,2.0,13.0,14.0,12.0,7.0,1.0,5.0] |1.0  |
|40CD7

In [26]:
df_final.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('f1', 'float'),
 ('f2', 'float'),
 ('f3', 'float'),
 ('f4', 'float'),
 ('f5', 'float'),
 ('f6', 'float'),
 ('f7', 'float'),
 ('f8', 'float'),
 ('f9', 'float'),
 ('f10', 'float'),
 ('f11', 'float'),
 ('f12', 'float'),
 ('features', 'vector'),
 ('label', 'double')]

In [27]:
trainDF,testDF = df_final.randomSplit([0.7,0.3],seed=0)

In [28]:
trainDF.count()

135846957

# Training

In [1]:
df_final = spark.read.parquet('/data/user/hive/warehouse/ian/feature/mac_features')
trainDF,testDF = df_final.randomSplit([0.7,0.3],seed=0)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import numpy as np

rf = RandomForestClassifier()

paramGrid_rf = ParamGridBuilder()\
       .addGrid(rf.maxDepth,[3,4,5,6,7,8,9,10])\
       .addGrid(rf.numTrees,[10,15,20,25,30])\
       .build()

crossval_rf = CrossValidator(estimator=rf,
                            estimatorParamMaps=paramGrid_rf,
                            evaluator=MulticlassClassificationEvaluator(),
                            numFolds=5)

cvModel_rf = crossval_rf.fit(trainDF)

In [ ]:
best_model_rf = cvModel_rf.bestModel

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_rf = BinaryClassificationEvaluator(
    rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_rf.evaluate(best_model_rf.transform(testDF))